In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import string
import re

from tqdm import tqdm

In [0]:
import json
from pandas.io.json import json_normalize

In [0]:
def mapping_label(x):
      return lab['label'][x]

In [0]:
def get_labels_df(my_json, lbls):
    data = []
    for line in open(my_json, 'r',errors='ignore'):
        data.append(json.loads(line))
    usr_list=[]
    for i in range(len(data)):
        for j in data[i]['annotations']:
            usr_list.append(j['user'])
    
    usr_dict = {}
    for i in set(usr_list):
        usr_dict[i]=[]
    
    for i in range(len(data)):
        for k,v in usr_dict.items():
            usr_dict[k] = []
            for j in range(len(data[i]['annotations'])):
                if data[i]['annotations'][j]['user']==k:
                    usr_dict[k].append(data[i]['annotations'][j]['label'])
                data[i][k]=usr_dict[k]
    labels=lbls
    lab = pd.read_csv(labels)
    lab = lab.to_dict()
    lab_dict = dict(zip([lab['number'][j] for j in lab['number']], [lab['label'][j] for j in lab['label']]))
    for k,v in usr_dict.items():
        for i in range(len(data)):
            try:
                data[i][k]= [ lab_dict.get(item,item) for item in data[i][k]]
            except:
                pass
    df = pd.DataFrame.from_dict(data)
    unique_users = list(set(usr_list))
    df['lab_final'] = [[]] * len(df)
    for user in unique_users:
        for i in range(len(df)):
            if df[user][i]:
                df['lab_final'][i] = df[user][i]  
    df= df[df['lab_final'].str.len() != 0]
    df= df[df['lab_final']==df['lab_final']] 
    #-------------------------------------------
    print(list(usr_dict.keys()))
    df.drop(list(usr_dict.keys()), axis = 1, inplace=True)
    #-------------------------------------------  
    print(df.shape)
    return df

In [0]:
def perc_approved(df):
  print('Labels distribution')
  print(df.lab_final.apply(tuple).value_counts())
  print("\\------------------------------------")
  print('Percentage approved and percentage left:')
  print("Total rows:",df.shape[0])
  print("ISNULL, i.e Unlabeled ",df['lab_final'].isnull().sum())
  print("ISNULL, i.e. Not Approved ",df.annotation_approver.isnull().sum())
  approvers = list(set(df.annotation_approver))
  for name in approvers:
    if name != None:
        print(name,df[df.annotation_approver == name].shape[0])

## main_df
This is the parent file which contains all the features and from which the text was fed into doccano. <br />
It is used for joining/merging with extracted doccano files

In [0]:
main_df = pd.read_csv('sexual_violence_twitter-filtered.csv', keep_default_na=False)

In [0]:
main_df = pd.read_csv('DV_tweets_1500.csv', keep_default_na=False)

In [0]:
main_df = pd.read_csv('India_English_Words_Filtered.csv', keep_default_na=False)

In [0]:
main_df = pd.read_csv('twitterDataEnglish.csv', keep_default_na=False)

In [11]:
main_df.shape

(3070, 20)

In [12]:
main_df.head(2)

,Unnamed: 0,username,to,text,retweets,favorites,replies,id,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls,user_location,month,processed_text,label
0,0,VipinHeartsTay,,The fact their might be so many marital rape a...,0,10,1,1255927001006903297,https://twitter.com/VipinHeartsTay/status/1255...,2875165356,2020-04-30 18:28:25+00:00,Thu Apr 30 18:28:25 +0000 2020,,,,,India,4,fact might many marital rape sexual assault wi...,DV
1,1,TheGhoudi,,"‘If the child has been through sexual abuse, a...",2,3,1,1255805741769715712,https://twitter.com/TheGhoudi/status/125580574...,822838819359690754,2020-04-30 10:26:35+00:00,Thu Apr 30 10:26:35 +0000 2020,,@TanujaSodhi,,,"Mumbai, India",4,"‘if child sexual abuse, adults/parents give ch...",DV


In [13]:
main_df.duplicated(['text']).any()

False

In [0]:
#tweets = main_df['text']
#main_df[tweets.isin(tweets[tweets.duplicated()])]

In [0]:
#main_df.drop_duplicates(subset='text',inplace = True)

## Get dataframe from downloaded doccano json file

### dvclasses project 14 file

In [0]:
my_json="dvclasses_proj_14.json"
lbls='dvclassesLabels.csv'

In [0]:
df = get_labels_df(my_json,lbls)

(353, 13)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
df.lab_final = df[35]

In [0]:
perc_approved(df)

Labels distribution
(NON-D_VIOLENCE_ABOUT,)       127
(NO_VIOLENCE,)                 91
(DV_OPINION_ADVOCATE,)         69
()                             21
(DV_STORY,)                    17
(DV_OPINIO _DENY,)             11
(DV_OPINION_INFO_NEWS,)         9
(NON_D_VIOLENCE_DIRECTED,)      8
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 353
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  344
anel 9


In [0]:
df.head(1)

,id,text,annotations,meta,annotation_approver,33,34,35,40,41,26,27,lab_final
0,11316,A university lecturer in Korat has been charge...,"[{'label': 75, 'user': 33}, {'label': 75, 'use...",{},anel,[DV_OPINION_INFO_NEWS],[DV_OPINION_INFO_NEWS],[DV_OPINION_INFO_NEWS],[DV_OPINION_INFO_NEWS],[DV_STORY],[],[],[DV_OPINION_INFO_NEWS]


In [0]:
df.drop(['meta',33,34,35,40,41,26,27], axis = 1, inplace=True) 

In [0]:
df.shape

(353, 5)

Converting **lab_final** from list type column to text

In [0]:
df.lab_final[5][0]

'NO_VIOLENCE'

In [0]:
df['lab_final'] = df['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [0]:
df.duplicated(['text']).any()

True

In [0]:
tweets = df['text']
df[tweets.isin(tweets[tweets.duplicated()])]

,id,text,annotations,annotation_approver,lab_final
184,11361,"Coronavirus: Jobless man, pregnant wife forced...","[{'label': 75, 'user': 40}, {'label': 75, 'use...",None,NON-D_VIOLENCE_ABOUT
185,11362,"Coronavirus: Jobless man, pregnant wife forced...","[{'label': 71, 'user': 34}, {'label': 75, 'use...",None,NON-D_VIOLENCE_ABOUT


In [0]:
df.lab_final.iloc[184] = 'NON-D_VIOLENCE_ABOUT'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


**Inner join / Merge**

First checking how many common entries there are in both the dfs, through looping, because inner join gave very few results

In [0]:
df['text'] = df['text'].str.lower()
main_df['text'] = main_df['text'].str.lower()

In [0]:
#count = 0
#for row in range(len(main_df)):
#  for r in range(len(df)):
#    if main_df.iloc[row].text == df.iloc[r].text:
#      count+=1
#print(count)

30


**Merge**

In [0]:
data_dvclasses = pd.merge(main_df, df, on='text', how='inner')

In [0]:
data_dvclasses.shape

(30, 17)

Saving as csv

In [0]:
df.to_csv('data_dvclasses.csv', index=False)

# data1.json

In [0]:
my_json="data1.json"
lbls='dvtweetsLabels.csv'

In [0]:
df = get_labels_df(my_json,lbls)

(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
df.head(2)

,id,text,annotations,annotation_approver,lab_final
0,11316,A university lecturer in Korat has been charge...,"[{'label': 75, 'user': 33}, {'label': 75, 'use...",anel,DV_OPINION_INFO_NEWS
1,11234,"Baba with the amount they have, they can use i...","[{'label': 76, 'user': 34}, {'label': 76, 'use...",anel,DV_OPINION_ADVOCATE


In [0]:
perc_approved(df)

Labels distribution
(N, O, N, -, D, _, V, I, O, L, E, N, C, E, _, A, B, O, U, T)             127
(N, O, _, V, I, O, L, E, N, C, E)                                         91
(D, V, _, O, P, I, N, I, O, N, _, A, D, V, O, C, A, T, E)                 69
()                                                                        21
(D, V, _, S, T, O, R, Y)                                                  17
(D, V, _, O, P, I, N, I, O,  , _, D, E, N, Y)                             11
(D, V, _, O, P, I, N, I, O, N, _, I, N, F, O, _, N, E, W, S)               9
(N, O, N, _, D, _, V, I, O, L, E, N, C, E, _, D, I, R, E, C, T, E, D)      8
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 353
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  344
anel 9


Rows left for Checking

In [0]:
#df[df.annotation_approver.isnull()]

**Inner join / Merge**

In [0]:
data1 = pd.merge(main_df, df, on="text", how='inner')

In [0]:
data1.shape

(251, 25)

In [0]:
data1.head(1)

,Unnamed: 0,username,to,text,retweets,favorites,replies,id_x,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls,user_location,month,processed_text,label,id_y,annotations,meta,annotation_approver,lab_final
0,0,VipinHeartsTay,,The fact their might be so many marital rape a...,0,10,1,1255927001006903297,https://twitter.com/VipinHeartsTay/status/1255...,2875165356,2020-04-30 18:28:25+00:00,Thu Apr 30 18:28:25 +0000 2020,,,,,India,4,fact might many marital rape sexual assault wi...,DV,9213,"[{'label': 24, 'user': 1}]",{},admin,[DV_OPINION_ADVOCATE]


In [0]:
data1.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True) 

Converting **lab_final** from a list type column to text

In [0]:
data1.lab_final[5][0]

'N'

In [0]:
data1['lab_final'] = data1['lab_final'].apply(lambda x: ','.join(map(str, x)))

Saving csv

In [0]:
data1.to_csv('data1_dvtweets.csv', index=False)

# data2.json

In [0]:
my_json="data2.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       157
(NO_VIOLENCE,)                 70
(NON_D_VIOLENCE_DIRECTED,)     49
(DV_OPINION_ADVOCATE,)         11
(DV_STORY,)                     7
(DV_OPINION_NEWS_INFO,)         5
(DV_OPINION_DENIER,)            1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 300
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  0
tim 1
admin 299


In [0]:
#df[df.annotation_approver.isnull()]

,id,text,annotations,meta,annotation_approver,lab_final


In [0]:
data2 = pd.merge(main_df, df, on="text", how='inner')
print(data2.shape)
data2.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True) 

(246, 25)


In [0]:
data2.head(1)

,username,to,text,retweets,favorites,replies,id_x,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls,user_location,month,processed_text,label,annotations,annotation_approver,lab_final
0,incrediblerer,,SAY NO TO SEXUAL VIOLENCE Please punish perpet...,0,0,0,1242503166811189249,https://twitter.com/incrediblerer/status/12425...,894800077142728704,2020-03-24 17:26:54+00:00,Tue Mar 24 17:26:54 +0000 2020,#AV #nthroom_stop #nthroom_case,,,,,3,say sexual violence please punish perpetrator ...,DV,"[{'label': 48, 'user': 1}]",admin,[NON_D_VIOLENCE_ABOUT]


In [0]:
data2.lab_final[5][0]

'N'

In [0]:
data2['lab_final'] = data2['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [0]:
data2.to_csv('data2_dvtweets.csv', index=False)

# data3.json

In [0]:
my_json="data3.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(274, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       162
(NO_VIOLENCE,)                 65
(NON_D_VIOLENCE_DIRECTED,)     36
(DV_OPINION_ADVOCATE,)          5
(DV_OPINION_NEWS_INFO,)         3
(DV_STORY,)                     2
(UNKNOWN,)                      1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 274
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  229
javier 45


**Check df rows with two values of df.lab_final** 

In [0]:
np.where(df['lab_final'].apply(lambda x: x == ['NON_D_VIOLENCE_DIRECTED', 'NON_D_VIOLENCE_ABOUT']))

(array([178]),)

In [0]:
df.iloc[178]

id                                                                 10016
text                   New episode of #nofilterfriday on @publichouse...
annotations            [{'label': 53, 'user': 11}, {'label': 52, 'use...
meta                                                                  {}
annotation_approver                                                 None
lab_final                [NON_D_VIOLENCE_DIRECTED, NON_D_VIOLENCE_ABOUT]
Name: 199, dtype: object

In [0]:
df['lab_final'].iloc[178] = ['NON_D_VIOLENCE_ABOUT']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
#df[df.annotation_approver.isnull()]

**Merging / Joining**

In [0]:
data3 = pd.merge(main_df, df, on="text", how='inner')
print(data3.shape)
data3.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True)

(236, 25)


In [0]:
data3.lab_final[5][0]

'N'

In [0]:
data3['lab_final'] = data3['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [0]:
data3.to_csv('data3_dvtweets.csv', index=False)

# data4.json

In [60]:
my_json="data4.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

[1, 19, 9, 7]
(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [61]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       126
(NON_D_VIOLENCE_DIRECTED,)     79
(NO_VIOLENCE,)                 57
(DV_OPINION_ADVOCATE,)         22
(DV_OPINION_NEWS_INFO,)         6
(UNKNOWN,)                      6
(DV_OPINION_DENIER,)            2
(DV_STORY,)                     2
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 300
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  1
admin 39
thiago 260


In [0]:
#df[df.annotation_approver.isnull()]

In [63]:
data4 = pd.merge(main_df, df, on="text", how='inner')
print(data4.shape)
data4.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True)

(256, 25)


In [64]:
data4.lab_final[5][0]

'NON_D_VIOLENCE_ABOUT'

In [0]:
data4['lab_final'] = data4['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [66]:
data4.columns

Index(['username', 'to', 'text', 'retweets', 'favorites', 'replies', 'id_x',
       'permalink', 'author_id', 'date', 'formatted_date', 'hashtags',
       'mentions', 'geo', 'urls', 'user_location', 'month', 'processed_text',
       'label', 'annotations', 'annotation_approver', 'lab_final'],
      dtype='object')

In [0]:
data4.to_csv('data4_dvtweets.csv', index=False)

# data5.json

In [0]:
my_json="data5.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       212
(NO_VIOLENCE,)                 67
(DV_OPINION_ADVOCATE,)          9
(NON_D_VIOLENCE_DIRECTED,)      8
(DV_OPINION_NEWS_INFO,)         2
(UNKNOWN,)                      1
(DV_STORY,)                     1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 300
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  1
tim 219
admin 80


In [0]:
#df[df.annotation_approver.isnull()]

In [0]:
data5 = pd.merge(main_df, df, on="text", how='inner')
print(data5.shape)
data5.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True)

(264, 25)


In [0]:
data5.lab_final[5][0]

'N'

In [0]:
data5['lab_final'] = data5['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [0]:
data5.to_csv('data5_dvtweets.csv', index=False)

# data6.json

In [68]:
my_json="data6.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

[1, 22]
(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [69]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       217
(NO_VIOLENCE,)                 57
(NON_D_VIOLENCE_DIRECTED,)     14
(DV_OPINION_ADVOCATE,)          9
(DV_OPINION_NEWS_INFO,)         2
(DV_STORY,)                     1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 300
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  2
admin 298


In [0]:
#df[df.annotation_approver.isnull()]

In [71]:
data6 = pd.merge(main_df, df, on="text", how='inner')
print(data6.shape)
data6.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True)

(267, 25)


In [72]:
data6.lab_final[5][0]

'NON_D_VIOLENCE_ABOUT'

In [0]:
data6['lab_final'] = data6['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [74]:
data6.columns

Index(['username', 'to', 'text', 'retweets', 'favorites', 'replies', 'id_x',
       'permalink', 'author_id', 'date', 'formatted_date', 'hashtags',
       'mentions', 'geo', 'urls', 'user_location', 'month', 'processed_text',
       'label', 'annotations', 'annotation_approver', 'lab_final'],
      dtype='object')

In [0]:
data6.to_csv('data6_dvtweets.csv', index=False)

# data7.json

In [0]:
my_json="data7.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(298, 7)
(NON_D_VIOLENCE_ABOUT,)       207
(NO_VIOLENCE,)                 46
(NON_D_VIOLENCE_DIRECTED,)     24
(UNKNOWN,)                     12
(DV_OPINION_NEWS_INFO,)         7
(DV_OPINION_ADVOCATE,)          1
(DV_STORY,)                     1
Name: lab_final, dtype: int64
ISNULL Label  0
ISNULL Approver  298
Total rows: 298


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

^This queue is not approved

In [0]:
#df[df.annotation_approver.isnull()]

In [0]:
df.to_csv('data7_dvtweets.csv', index=False)

# data8.json

In [0]:
my_json="data8.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(300, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)       215
(NO_VIOLENCE,)                 46
(NON_D_VIOLENCE_DIRECTED,)     21
(DV_OPINION_NEWS_INFO,)         6
(DV_STORY,)                     5
(DV_OPINION_ADVOCATE,)          4
(DV_OPINION_DENIER,)            2
(UNKNOWN,)                      1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 300
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  3
javier 13
admin 284


In [0]:
#df[df.annotation_approver.isnull()]

In [0]:
data8 = pd.merge(main_df, df, on="text", how='inner')
print(data1.shape)
data8.drop(['Unnamed: 0','id_y','meta'], axis = 1, inplace=True)

(251, 22)


In [0]:
data8.lab_final[5][0]

'N'

In [0]:
data8['lab_final'] = data8['lab_final'].apply(lambda x: ','.join(map(str, x)))

In [0]:
data8.to_csv('data8_dvtweets.csv', index=False)

# data9.json

In [0]:
my_json="data9.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(186, 8)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

Labels distribution
(NON_D_VIOLENCE_ABOUT,)                        114
(NO_VIOLENCE,)                                  55
(NON_D_VIOLENCE_DIRECTED,)                       7
(DV_OPINION_ADVOCATE,)                           5
(DV_OPINION_NEWS_INFO,)                          2
(DV_STORY,)                                      2
(DV_OPINION_ADVOCATE, NON_D_VIOLENCE_ABOUT)      1
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 186
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  170
admin 16


In [0]:
#df[df.annotation_approver.isnull()]

In [0]:
df.to_csv('data9_dvtweets.csv', index=False)

# data10.json

In [0]:
my_json="data10.json"
lbls='dvtweetsLabels.csv'
df = get_labels_df(my_json,lbls)

(365, 8)
(NON_D_VIOLENCE_ABOUT,)       191
(NO_VIOLENCE,)                122
(NON_D_VIOLENCE_DIRECTED,)     26
(DV_OPINION_ADVOCATE,)         18
(DV_STORY,)                     3
(DV_OPINION_DENIER,)            3
(DV_OPINION_NEWS_INFO,)         2
Name: lab_final, dtype: int64
ISNULL Label  0
ISNULL Approver  358
Total rows: 365
admin 7


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
perc_approved(df)

In [0]:
#df[df.annotation_approver.isnull()]

In [0]:
df.to_csv('data10_dvtweets.csv', index=False)

### Combine all csvs

In [0]:
all_filenames = ['data1_dvtweets.csv','data2_dvtweets.csv','data3_dvtweets.csv','data4_dvtweets.csv', 'data5_dvtweets.csv','data6_dvtweets.csv','data8_dvtweets.csv']

In [0]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "label_approved.csv", index=False, encoding='utf-8-sig')

In [0]:
labeled = pd.read_csv('label_approved.csv', keep_default_na=False)

In [79]:
print(type(labeled.at[0, 'lab_final']))

<class 'str'>


In [80]:
perc_approved(labeled)

Labels distribution
(N, O, N, _, D, _, V, I, O, L, E, N, C, E, _, A, B, O, U, T)             1089
(N, O, _, V, I, O, L, E, N, C, E)                                         374
(N, O, N, _, D, _, V, I, O, L, E, N, C, E, _, D, I, R, E, C, T, E, D)     191
(D, V, _, O, P, I, N, I, O, N, _, A, D, V, O, C, A, T, E)                  61
(D, V, _, O, P, I, N, I, O, N, _, N, E, W, S, _, I, N, F, O)               20
(D, V, _, O, P, I, N, I, O, N, _, I, N, F, O, _, N, E, W, S)               17
(D, V, _, S, T, O, R, Y)                                                   16
(U, N, K, N, O, W, N)                                                       9
(D, V, _, O, P, I, N, I, O, N, _, D, E, N, I, E, R)                         5
Name: lab_final, dtype: int64
\------------------------------------
Percentage approved and percentage left:
Total rows: 1782
ISNULL, i.e Unlabeled  0
ISNULL, i.e. Not Approved  0
 211
tim 286
javier 51
thiago 222
admin 1012


In [81]:
# Checking type of data in lab_final
labeled.lab_final[:5]

0     DV_OPINION_ADVOCATE
1    NON_D_VIOLENCE_ABOUT
2    NON_D_VIOLENCE_ABOUT
3    NON_D_VIOLENCE_ABOUT
4             NO_VIOLENCE
Name: lab_final, dtype: object

In [0]:
#labeled.loc[labeled.lab_final == 'DV_OPINION_NEWS_INFO','lab_final'] = 'DV_OPINION_INFO_NEWS'

In [90]:
labeled.shape

(1782, 22)

In [84]:
labeled.lab_final.value_counts()

NON_D_VIOLENCE_ABOUT       1089
NO_VIOLENCE                 374
NON_D_VIOLENCE_DIRECTED     191
DV_OPINION_ADVOCATE          61
DV_OPINION_NEWS_INFO         20
DV_OPINION_INFO_NEWS         17
DV_STORY                     16
UNKNOWN                       9
DV_OPINION_DENIER             5
Name: lab_final, dtype: int64

Checking columns with **NULL** Values

In [85]:
labeled.isnull().sum()

username               0
to                     0
text                   0
retweets               0
favorites              0
replies                0
id_x                   0
permalink              0
author_id              0
date                   0
formatted_date         0
hashtags               0
mentions               0
geo                    0
urls                   0
user_location          0
month                  0
processed_text         0
label                  0
annotations            0
annotation_approver    0
lab_final              0
dtype: int64

In [87]:
labeled.columns

Index(['username', 'to', 'text', 'retweets', 'favorites', 'replies', 'id_x',
       'permalink', 'author_id', 'date', 'formatted_date', 'hashtags',
       'mentions', 'geo', 'urls', 'user_location', 'month', 'processed_text',
       'label', 'annotations', 'annotation_approver', 'lab_final'],
      dtype='object')

In [0]:
labeled = labeled.rename(columns={'id_x': 'tweet_id'})

In [0]:
labeled.to_csv('labeled.csv', index=False)